In [1]:
%pylab inline
import matplotlib.pyplot as plt 
import numpy as np
import cv2
np.seterr(all='raise')

Populating the interactive namespace from numpy and matplotlib


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def nd(l):
    return np.array(l,dtype=float)
def Vis(U):
# Visualizing A Cluster
    img=U/U.max()
    img=(img*255).astype(int)
    plt.imshow(img,cmap='gray')
    plt.show()
def VisCrisp(U):
    img=(U==numpy.amax(U,axis=(0)))*255
    for Map in img:
        Vis(Map)

In [3]:
def normalizeImage(img):
    minPx,maxPx=img.min(),img.max()
    NormalizedImg = (img-minPx)/(maxPx-minPx)
    return NormalizedImg

In [4]:
from scipy.signal import convolve2d
def conv(Mat,ngb):
    if ngb==0:
        return Mat
    ngbhd=2*ngb+1
    Filter=np.ones((ngbhd, ngbhd))
    Filter[ngb,ngb]=0
    if len(Mat.shape) == 2:
        return convolve2d(Mat, Filter, mode='same', boundary='fill', fillvalue=0)
    for i in range(Mat.shape[0]):
        Mat[i]=convolve2d(Mat[i], Filter, mode='same', boundary='fill', fillvalue=0)
    return Mat

def SNI(U,D2,ngb):
    return conv(U*D2,ngb)/conv(U,ngb)

def NebTerm(U,x,ngb):
    return conv(U*x,ngb)/conv(U,ngb)

In [5]:
def sugenoNeg(mu,b=0.7):
    return (1-mu)/(1+b*mu)

def randomUMatGen(clusters,img_shape):
    #U is the Uij matrix
    U = np.random.rand(clusters,img_shape[0],img_shape[1])
    U = U / U.sum(axis=0)
    return U

def randomBMatGen(img_shape):
    #B is the bias matrix
    B = np.random.rand(img_shape[0],img_shape[1])/1000000
    return B

def UpCen(U,x,alpha,ngb,m):
    T=x+alpha*NebTerm(U,x,ngb)
    #Um=putEps(np.power(U,m))
    Um=np.power(U,m)
    return (Um*T).sum(axis=(1,2)) / (Um.sum(axis=(1,2)) * (1+alpha) )

def getCentroids(X,U,alpha,ngb,m=2):
    #U=1-sugenoNeg(U)
    V= UpCen(U,X,alpha,ngb,m)
    return V

def d2(X,Y):
    return (np.power(X-Y,2))

def updateMatU(X,V,U,alpha,ngb,m):
    delta = float("0.1e-7")
    D2=d2(X[np.newaxis,:,:],V[:,np.newaxis,np.newaxis])
    D2=D2+SNI(U,D2,ngb)+delta
    D2=np.power(D2,-1/(m-1))
    D2=D2/D2.sum(axis=0)
    #D2=putEps(D2)
    return D2

from scipy import ndimage
def GaussianSmoothing(Img,sigma=0.01):
    return ndimage.filters.gaussian_filter(Img, sigma, mode='constant')


def getB(X,U,V,Alpha,m,ngb):
    Um=np.power(U,m)
    #Um=putEps(Um)
    Beta=Um+Alpha*conv(Um,ngb)
    T1=(Beta*V[:,np.newaxis,np.newaxis]).sum(axis=0)/(Beta.sum(axis=0))
    T1=2*(X-T1)
    T1=T1.sum()
    T2=1/Beta.sum(axis=0)
    T2=T2.sum()
    Lambda=T1/T2
    #Lambda=putEps(Lambda)
    B1= (Beta*V[:,np.newaxis,np.newaxis]).sum(axis=0) + Lambda/2
    B1=B1/(Beta.sum(axis=0))
    B=X-B1
    return GaussianSmoothing(B)

In [13]:
def fcm(mu,alpha=0.2,ngb=1,m=2,c=4,e=0.0001,maxIter=400):
    X=nd(mu)
    U=randomUMatGen(c,mu.shape)
    V=getCentroids(X,U,alpha,ngb,m)
    B=randomBMatGen(X.shape)
    B=0*B

    ind=1
    U_prev=zeros_like(U)
    while ind<maxIter:
        if (np.absolute(U-U_prev)).max() < e:
            break
        U_prev=U    
        U=updateMatU(X-B,V,U,alpha,ngb,m)
        V=getCentroids(X-B,U,alpha,ngb,m)
        #B=getB(X,U,V,alpha,m,ngb)
        ind+=1
    print(ind)
    return U,V    

In [11]:
def printImg(img):
    plt.imshow(img,cmap='gray')
    plt.show()
    
def GetImg(FileName):
    import os
    cwd = os.getcwd()
    img = cv2.imread(cwd+'\\Img\\'+FileName, cv2.IMREAD_GRAYSCALE)
    return img

def GetMap(U):
    Map=(U==numpy.amax(U,axis=(0))).astype(int)
    return Map

def GetGroundTurthMaps(FileName='NewImg2.tiff'):
    import os
    cwd = os.getcwd()
    img1 = GetImg(FileName)
    uniq=np.unique(img1)
    uniq.sort()
    Map=[]
    for i in range(uniq.size):
        Map.append(img1==uniq[i])
    return nd(Map),uniq,img1

def GetResMaps(res,getGrndTruth=0):
    img=GetImg('NewImg2.tiff')
    Maps=GetMap(res[0])
    Maps=nd(Maps)
    Avg=nd([(img*Map).sum()/Map.sum() for Map in Maps])
    Ord=numpy.argsort(Avg)
    Maps=Maps[Ord]
    #Maps=Maps*Inten[:,np.newaxis,np.newaxis] #To get the colored resultant
    return Maps

In [12]:
def DiceScore(G,R):
    intersectionPx=2*(G*R).sum(axis=(1,2))
    return intersectionPx/(G+R).sum(axis=(1,2))
def AvgSegAcc(G,R):
    intersectionPx=(G*R).sum()
    return intersectionPx/G.sum()
def Scores(G,R):
    return DiceScore(G,R),AvgSegAcc(G,R)

def GetScores(FileName,G,printResMap=1):
    res=fcm(GetImg(FileName))
    R=GetResMaps(res)
    if printResMap:
        for seg in R:
            printImg(seg)
    return Scores(G,R)

def GetAllImgScores(printResMap=0):
    import os
    path = os.getcwd()+'\\Img'
    Files=os.listdir(path)
    G,a,b=GetGroundTurthMaps()
    FilesScr={}
    for file in Files:
        FilesScr[file]=GetScores(file,G,printResMap)
        print(FilesScr[file])
    return FilesScr

In [9]:
GetAllImgScores()

400
(array([1., 1., 1., 1.]), 1.0)
400
(array([1.        , 1.        , 0.99989308, 0.99998043]), 0.9999738930659984)
400
(array([7.76781042e-01, 3.06896416e-01, 2.35349494e-04, 7.94571340e-01]), 0.5890246449456976)
400
(array([0.99990377, 0.99994895, 0.99890939, 0.99977694]), 0.999702380952381)
400
(array([0.99985567, 0.99992342, 0.9986529 , 0.99971823]), 0.999624060150376)


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([1.        , 1.        , 0.99989308, 0.99998043]),
  0.9999738930659984),
 'NewImg2_0_002.tiff': (array([7.76781042e-01, 3.06896416e-01, 2.35349494e-04, 7.94571340e-01]),
  0.5890246449456976),
 'NewImg2_0_003.tiff': (array([0.99990377, 0.99994895, 0.99890939, 0.99977694]),
  0.999702380952381),
 'NewImg2_0_004.tiff': (array([0.99985567, 0.99992342, 0.9986529 , 0.99971823]),
  0.999624060150376)}

In [14]:
GetAllImgScores()

400
(array([1., 1., 1., 1.]), 1.0)
400
(array([1.        , 1.        , 0.99989308, 0.99998043]), 0.9999738930659984)
400
(array([1.        , 0.99997447, 0.99918734, 0.99984738]), 0.9997963659147869)
400
(array([0.99990377, 0.99994895, 0.99890939, 0.99977694]), 0.999702380952381)
400
(array([0.99985567, 0.99992342, 0.9986529 , 0.99971823]), 0.999624060150376)


{'NewImg2.tiff': (array([1., 1., 1., 1.]), 1.0),
 'NewImg2_0_001.tiff': (array([1.        , 1.        , 0.99989308, 0.99998043]),
  0.9999738930659984),
 'NewImg2_0_002.tiff': (array([1.        , 0.99997447, 0.99918734, 0.99984738]),
  0.9997963659147869),
 'NewImg2_0_003.tiff': (array([0.99990377, 0.99994895, 0.99890939, 0.99977694]),
  0.999702380952381),
 'NewImg2_0_004.tiff': (array([0.99985567, 0.99992342, 0.9986529 , 0.99971823]),
  0.999624060150376)}